# Train a Linear Regression Model with Watson Machine Learning 

Notebook created by Zeming Zhao on June, 2021

In this notebook, you will learn how to use the Watson Machine Learning Accelerator (WML-A) API and accelerate the processing of Linear Regression model on GPU with Watson Machine Learning Accelerator.

Linear Regression is a simple machine learning model where the response y is modelled by a linear combination of the predictors in X.

In this notebook we have three versions of Linear Regression model: scikit-learn version, cuML version and snapML version.

All three versions will be submitted onto WMLA. And we can compare the performance benifit of cuML and snapML version.

This notebook covers the following sections:

1. [Generate dataset for Linear Regression](#dataset)<br>

1. [Setup Linear Regression using SK-Learning](#skl-model)<br>

1. [Training the SK-Learning model on CPU with Watson Machine Learning Accelerator](#skl-cpu)<br>

1. [Setup Linear Regression using cuML](#cuml-model)<br>

1. [Training the cuML model on GPU with Watson Machine Learning Accelerator](#cuml-gpu)<br>

1. [Setup Linear Regression using snapML](#snapml-model)<br>

1. [Training the SnapML model on CPU with Watson Machine Learning Accelerator](#snapml-cpu)<br>

1. [Setup Linear Regression using snapML GPU](#snapml-model-gpu)<br>

1. [Training the SnapML model on GPU with Watson Machine Learning Accelerator](#snapml-gpu)<br>

## Preparations
### Prepare directory and file for writing Linear Regression engine.

In [1]:
from pathlib import Path
dataset_dir = f'/project_data/data_asset/dataset' 
dataset_base_name = f'linearRegression-dataset.py'
Path(dataset_dir).mkdir(exist_ok=True)
print("create dataset directory done.")

create dataset directory done.


In [2]:
from pathlib import Path
model_dir = f'/project_data/data_asset/models' 
model_base_name = f'linearRegression-main.py'
Path(model_dir).mkdir(exist_ok=True)
print("create model directory done.")

create model directory done.


<a id = "dataset"></a>
## Step 1 : Generate dataset for Linear Regression

In [3]:
dataset_main='linearregression-'+dataset_base_name

In [4]:
%%writefile {dataset_dir}/{dataset_main}

import os, datetime
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_sparse_uncorrelated
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression as skLinearRegression
import numpy as np
from scipy.sparse import csr_matrix
from scipy.sparse import save_npz
from pathlib import Path

# Define Parameters for a large regression
n_samples = 2**19 #If you are running on a GPU with less than 16GB RAM, please change to 2**19 or you could run out of memory
n_features = 699
random_state = 42

# Generate Data
start = datetime.datetime.now()
X, y = make_sparse_uncorrelated(n_samples=n_samples, n_features=n_features, random_state=random_state)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=random_state)
end = datetime.datetime.now()
print ("generate data timecost: %.2gs" % ((end-start).total_seconds()))

print("Number of examples: %d" % (X_train.shape[0]))
print("Number of features: %d" % (X_train.shape[1]))

data_dir =  "/gpfs/mydatafs/datasets/dataset-lr"
Path(data_dir).mkdir(exist_ok=True)
start = datetime.datetime.now()
save_npz("{0}/X_train.npz".format(data_dir),csr_matrix(X_train))
save_npz("{0}/X_test.npz".format(data_dir), csr_matrix(X_test))
end = datetime.datetime.now()
print ("convert samples to csr matrix and save, timecost: %.2gs" % ((end-start).total_seconds()))
start = datetime.datetime.now()
np.save("{0}/X_train.npy".format(data_dir),X_train) # for cuML only which does not support csr_matrix
np.save("{0}/X_test.npy".format(data_dir), X_test)  # for cuML only which does not support csr_matrix
end = datetime.datetime.now()
print ("save samples as numpy array for cuML, timecost: %.2gs" % ((end-start).total_seconds()))
start = datetime.datetime.now()
np.save("{0}/y_train.npy".format(data_dir), y_train)
np.save("{0}/y_test.npy".format(data_dir), y_test)
end = datetime.datetime.now()
print ("save labels as numpy array, timecost: %.2gs" % ((end-start).total_seconds()))

print("Genearte datasets done:")
print(*Path(data_dir).iterdir(), sep="\n")

Overwriting /project_data/data_asset/dataset/linearregression-linearRegression-dataset.py


### Prepare the model lib for job submission

In [5]:
import requests
from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

from matplotlib import pyplot as plt
%pylab inline

import base64
import json
import time
import urllib

Populating the interactive namespace from numpy and matplotlib


### Configuring your environment and project details
To set up your project details, provide your credentials in this cell. You must include your cluster URL, username, and password.

In [6]:
# please enter Watson Machine Learning Accelerator host name
hostname='wmla-console-wmla.apps.dse-perf.cpolab.ibm.com'
# login='username:password' # please enter the login and password
login='mluser1:mluser1'
es = base64.b64encode(login.encode('utf-8')).decode("utf-8")
# print(es)
commonHeaders={'Authorization': 'Basic '+es}
req = requests.Session()
auth_url = 'https://{}/auth/v1/logon'.format(hostname)
print(auth_url)

a=requests.get(auth_url,headers=commonHeaders, verify=False)
access_token=a.json()['accessToken']
# print("Access_token: ", access_token)

dl_rest_url = 'https://{}/platform/rest/deeplearning/v1'.format(hostname)
commonHeaders={'accept': 'application/json', 'X-Auth-Token': access_token}
req = requests.Session()

# Health check
confUrl = 'https://{}/platform/rest/deeplearning/v1/conf'.format(hostname)
r = req.get(confUrl, headers=commonHeaders, verify=False)

https://wmla-console-wmla.apps.dse-perf.cpolab.ibm.com/auth/v1/logon


### Define the status checking function

In [7]:
import tarfile
import tempfile
import os
import json
import pprint
import pandas as pd
from IPython.display import clear_output

def query_job_status(job_id,refresh_rate=3) :

    execURL = dl_rest_url  +'/execs/'+ job_id['id']
    pp = pprint.PrettyPrinter(indent=2)

    keep_running=True
    res=None
    while(keep_running):
        res = req.get(execURL, headers=commonHeaders, verify=False)
        monitoring = pd.DataFrame(res.json(), index=[0])
        pd.set_option('max_colwidth', 120)
        clear_output()
        print("Refreshing every {} seconds".format(refresh_rate))
        display(monitoring)
        pp.pprint(res.json())
        if(res.json()['state'] not in ['PENDING_CRD_SCHEDULER', 'SUBMITTED','RUNNING']) :
            keep_running=False
        time.sleep(refresh_rate)
    return res

### Define the submission function

In [8]:
def submit_job_to_wmla (args, files) :
    starttime = datetime.datetime.now()
    r = requests.post(dl_rest_url+'/execs?args='+args, files=files,
                  headers=commonHeaders, verify=False)
    if not r.ok:
        print('submit job failed: code=%s, %s'%(r.status_code, r.content))
    job_status = query_job_status(r.json(),refresh_rate=5)
    endtime = datetime.datetime.now()
    print("\nTotallly training cost: ", (endtime - starttime).seconds, " seconds.")

<a id = "cpu"></a>
### Define the submission parameters for scikit-learn version on cpu

In [9]:
# specify the model file, conda env, device type and device number
args = '--exec-start tensorflow --cs-datastore-meta type=fs \
--workerDeviceNum 1 \
--workerMemory 32G \
--workerDeviceType cpu \
--conda-env-name dlipy3-cpu  \
--model-main ' + dataset_main

print(args)

--exec-start tensorflow --cs-datastore-meta type=fs --workerDeviceNum 1 --workerMemory 32G --workerDeviceType cpu --conda-env-name dlipy3-cpu  --model-main linearregression-linearRegression-dataset.py


### Submit WMLA Workload

In [10]:
files = {'file': open("{0}/{1}".format(dataset_dir,dataset_main),'rb')}
submit_job_to_wmla (args, files)

Refreshing every 5 seconds


,id,args,submissionId,creator,state,appId,schedulerUrl,modelFileOwnerName,workDir,appName,createTime,elastic,nameSpace,numWorker,framework
0,wmla-1271,--exec-start tensorflow --cs-datastore-meta type=fs --workerDeviceNum 1 --workerMemory 32G --workerDeviceType cpu --...,wmla-1271,mluser1,FINISHED,wmla-1271,https://wmla-mss:9080,wmla,/gpfs/myresultfs/mluser1/batchworkdir/wmla-1271/_submitted_code,SingleNodeTensorflowTrain,2021-08-06T07:53:12Z,False,wmla,1,tensorflow


{ 'appId': 'wmla-1271',
  'appName': 'SingleNodeTensorflowTrain',
  'args': '--exec-start tensorflow --cs-datastore-meta type=fs '
          '--workerDeviceNum 1 --workerMemory 32G --workerDeviceType cpu '
          '--conda-env-name dlipy3-cpu  --model-main '
          'linearregression-linearRegression-dataset.py ',
  'createTime': '2021-08-06T07:53:12Z',
  'creator': 'mluser1',
  'elastic': False,
  'framework': 'tensorflow',
  'id': 'wmla-1271',
  'modelFileOwnerName': 'wmla',
  'nameSpace': 'wmla',
  'numWorker': 1,
  'schedulerUrl': 'https://wmla-mss:9080',
  'state': 'FINISHED',
  'submissionId': 'wmla-1271',
  'workDir': '/gpfs/myresultfs/mluser1/batchworkdir/wmla-1271/_submitted_code'}

Totallly training cost:  213  seconds.


<a id = "skl-model"></a>
## Step 2 : Setup Linear Regression using SK-Learning

In [11]:
model_main='sklean-'+model_base_name

In [12]:
%%writefile {model_dir}/{model_main}

import os, datetime
#from sklearn.datasets import make_regression
#from sklearn.model_selection import train_test_split
#from sklearn.datasets import make_sparse_uncorrelated
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression as skLinearRegression
import numpy as np
from scipy.sparse import load_npz

# load Data
data_dir =  "/gpfs/mydatafs/datasets/dataset-lr"
start = datetime.datetime.now()
X_train = load_npz(data_dir + "/X_train.npz")
X_test  = load_npz(data_dir + "/X_test.npz")
y_train = np.load(data_dir + "/y_train.npy")
y_test = np.load(data_dir + "/y_test.npy")
end = datetime.datetime.now()
print ("load data timecost: %.2gs" % ((end-start).total_seconds()))

print(type(X_train))
print("Number of examples: %d" % (X_train.shape[0]))
print("Number of features: %d" % (X_train.shape[1]))

# scikit-learn Model
lr_model = skLinearRegression(fit_intercept=True,
                            normalize=True,
                            n_jobs=-1)

# Fit
start = datetime.datetime.now()
lr_model.fit(X_train, y_train)
end = datetime.datetime.now()
print ("train timecost: %.2gs" % ((end-start).total_seconds()))

# Predict
start = datetime.datetime.now()
predict = lr_model.predict(X_test)
end = datetime.datetime.now()
print ("predict timecost: %.2gs" % ((end-start).total_seconds()))

# Evaluate
start = datetime.datetime.now()
r2_score = r2_score(y_test, predict)
end = datetime.datetime.now()
print ("evaluate timecost: %.2gs" % ((end-start).total_seconds()))

#  print("R^2 score (SKL):  %s" % r2_score_sk)
print("R^2 score (scikit-learn): %.4f" % r2_score)

Overwriting /project_data/data_asset/models/sklean-linearRegression-main.py


<a id = "skl-cpu"></a>
## Step 3 :  Training the SK-Learning model on CPU with Watson Machine Learning Accelerator


In [13]:
# specify the model file, conda env, device type and device number
args = '--exec-start tensorflow --cs-datastore-meta type=fs \
--workerDeviceNum 1 \
--workerMemory 16G \
--workerDeviceType cpu \
--conda-env-name dlipy3-cpu  \
--model-main ' + model_main

print(args)

--exec-start tensorflow --cs-datastore-meta type=fs --workerDeviceNum 1 --workerMemory 16G --workerDeviceType cpu --conda-env-name dlipy3-cpu  --model-main sklean-linearRegression-main.py


In [14]:
files = {'file': open("{0}/{1}".format(model_dir,model_main),'rb')}
submit_job_to_wmla (args, files)

Refreshing every 5 seconds


,id,args,submissionId,creator,state,appId,schedulerUrl,modelFileOwnerName,workDir,appName,createTime,elastic,nameSpace,numWorker,framework
0,wmla-1272,--exec-start tensorflow --cs-datastore-meta type=fs --workerDeviceNum 1 --workerMemory 16G --workerDeviceType cpu --...,wmla-1272,mluser1,FINISHED,wmla-1272,https://wmla-mss:9080,wmla,/gpfs/myresultfs/mluser1/batchworkdir/wmla-1272/_submitted_code,SingleNodeTensorflowTrain,2021-08-06T07:56:47Z,False,wmla,1,tensorflow


{ 'appId': 'wmla-1272',
  'appName': 'SingleNodeTensorflowTrain',
  'args': '--exec-start tensorflow --cs-datastore-meta type=fs '
          '--workerDeviceNum 1 --workerMemory 16G --workerDeviceType cpu '
          '--conda-env-name dlipy3-cpu  --model-main '
          'sklean-linearRegression-main.py ',
  'createTime': '2021-08-06T07:56:47Z',
  'creator': 'mluser1',
  'elastic': False,
  'framework': 'tensorflow',
  'id': 'wmla-1272',
  'modelFileOwnerName': 'wmla',
  'nameSpace': 'wmla',
  'numWorker': 1,
  'schedulerUrl': 'https://wmla-mss:9080',
  'state': 'FINISHED',
  'submissionId': 'wmla-1272',
  'workDir': '/gpfs/myresultfs/mluser1/batchworkdir/wmla-1272/_submitted_code'}

Totallly training cost:  63  seconds.


<a id = "cuml-model"></a>
## Step 4 :  Setup Linear Regression using cuML

In [15]:
model_main='cuML-'+model_base_name

In [16]:
%%writefile {model_dir}/{model_main}

import os, datetime
import cudf
#from cuml import make_regression, train_test_split
#from sklearn.datasets import make_sparse_uncorrelated
from cuml.linear_model import LinearRegression as cuLinearRegression
from cuml.metrics.regression import r2_score
import numpy as np
from scipy.sparse import load_npz
import pandas as pd
import gc

cudf.set_allocator("managed")

# specify the cache location to /gpfy since ~/.cache is not available
os.environ["CUPY_CACHE_DIR"]="/gpfs/mydatafs/models/cache/lr"

# load Data
data_dir =  "/gpfs/mydatafs/datasets/dataset-lr"
start = datetime.datetime.now()
X_train_ = load_npz(data_dir + "/X_train.npz")
print(X_train_.shape)
#X_train__ = pd.DataFrame.sparse.from_spmatrix(X_train_)
X_train = pd.DataFrame.sparse.from_spmatrix(X_train_)
del X_train_
gc.collect()
#X_train = cudf.DataFrame(X_train__[-200000:])
#del X_train__
#gc.collect()
X_test_ = load_npz(data_dir + "/X_test.npz")
print(X_test_.shape)
#X_test__ = pd.DataFrame.sparse.from_spmatrix(X_test_)
X_test = pd.DataFrame.sparse.from_spmatrix(X_test_)
del X_test_
gc.collect()
#X_test = cudf.DataFrame(X_test__[-50000:])
#del X_test__
#gc.collect()
end = datetime.datetime.now()
print ("load data timecost: %.2gs" % ((end-start).total_seconds()))

print("Number of examples: %d" % (X_train.shape[0]))
print("Number of features: %d" % (X_train.shape[1]))

start = datetime.datetime.now()
# X_train = np.load(data_dir + "/X_train.npy")
# X_test  = np.load(data_dir + "/X_test.npy")
y_train = np.load(data_dir + "/y_train.npy")
#y_train = y_train[-200000:]
y_test = np.load(data_dir + "/y_test.npy")
#y_test = y_test[-50000:]
end = datetime.datetime.now()
print ("load data timecost: %.2gs" % ((end-start).total_seconds()))

# cuML Model
lr_model = cuLinearRegression(fit_intercept=True,normalize=True,algorithm='eig')

# Fit
start = datetime.datetime.now()
lr_model.fit(X_train, y_train)
end = datetime.datetime.now()
print ("train timecost: %.2gs" % ((end-start).total_seconds()))

# Predict
start = datetime.datetime.now()
predict_cuml = lr_model.predict(X_test)
end = datetime.datetime.now()
print ("predict timecost: %.2gs" % ((end-start).total_seconds()))

# Evaluate
start = datetime.datetime.now()
r2_score_cuml = r2_score(y_test, predict_cuml)
end = datetime.datetime.now()
print ("evaluate timecost: %.2gs" % ((end-start).total_seconds()))

print("R^2 score (cuML): %.4f" % r2_score_cuml)

Overwriting /project_data/data_asset/models/cuML-linearRegression-main.py


<a id = "cuml-gpu"></a>
## Step 5 :  Training the cuML model on GPU with Watson Machine Learning Accelerator
### Re-define the submssion parameters

In [17]:
# specify the model file, conda env, device type and device number
args = '--exec-start tensorflow --cs-datastore-meta type=fs \
--workerDeviceNum 1 \
--workerMemory 16G \
--workerDeviceType gpu \
--conda-env-name rapids-21.06-new \
--model-main ' + model_main

print(args)

--exec-start tensorflow --cs-datastore-meta type=fs --workerDeviceNum 1 --workerMemory 16G --workerDeviceType gpu --conda-env-name rapids-21.06-new --model-main cuML-linearRegression-main.py


### Submit WMLA Workload

In [18]:
files = {'file': open("{0}/{1}".format(model_dir,model_main),'rb')}
submit_job_to_wmla (args, files)

Refreshing every 5 seconds


,id,args,submissionId,creator,state,appId,schedulerUrl,modelFileOwnerName,workDir,appName,createTime,elastic,nameSpace,numWorker,framework
0,wmla-1273,--exec-start tensorflow --cs-datastore-meta type=fs --workerDeviceNum 1 --workerMemory 16G --workerDeviceType gpu --...,wmla-1273,mluser1,FINISHED,wmla-1273,https://wmla-mss:9080,wmla,/gpfs/myresultfs/mluser1/batchworkdir/wmla-1273/_submitted_code,SingleNodeTensorflowTrain,2021-08-06T07:57:51Z,False,wmla,1,tensorflow


{ 'appId': 'wmla-1273',
  'appName': 'SingleNodeTensorflowTrain',
  'args': '--exec-start tensorflow --cs-datastore-meta type=fs '
          '--workerDeviceNum 1 --workerMemory 16G --workerDeviceType gpu '
          '--conda-env-name rapids-21.06-new --model-main '
          'cuML-linearRegression-main.py ',
  'createTime': '2021-08-06T07:57:51Z',
  'creator': 'mluser1',
  'elastic': False,
  'framework': 'tensorflow',
  'id': 'wmla-1273',
  'modelFileOwnerName': 'wmla',
  'nameSpace': 'wmla',
  'numWorker': 1,
  'schedulerUrl': 'https://wmla-mss:9080',
  'state': 'FINISHED',
  'submissionId': 'wmla-1273',
  'workDir': '/gpfs/myresultfs/mluser1/batchworkdir/wmla-1273/_submitted_code'}

Totallly training cost:  90  seconds.


<a id = "snapml-model"></a>
## Step 6:  Setup Linear Regression model using snapML

In [19]:
model_main='snapML-'+model_base_name

In [20]:
%%writefile {model_dir}/{model_main}

import datetime
#from sklearn.datasets import make_regression
#from sklearn.model_selection import train_test_split
#from sklearn.datasets import make_sparse_uncorrelated
from sklearn.metrics import r2_score
from snapml import LinearRegression as SnapLinearRegression
import numpy as np
from scipy.sparse import load_npz

# load Data
data_dir =  "/gpfs/mydatafs/datasets/dataset-lr"
start = datetime.datetime.now()
X_train = load_npz(data_dir + "/X_train.npz")
X_test  = load_npz(data_dir + "/X_test.npz")
y_train = np.load(data_dir + "/y_train.npy")
y_test = np.load(data_dir + "/y_test.npy")
end = datetime.datetime.now()
print ("load data timecost: %.2gs" % ((end-start).total_seconds()))

print("Number of examples: %d" % (X_train.shape[0]))
print("Number of features: %d" % (X_train.shape[1]))

# snapML model
lr_model = SnapLinearRegression(fit_intercept=True,normalize=True)

# Fit
start = datetime.datetime.now()
lr_model.fit(X_train, y_train)
end = datetime.datetime.now()
print ("train timecost: %.2gs" % ((end-start).total_seconds()))

r2_score_snapml = r2_score(y_test, lr_model.predict(X_test))
print("R^2 score (snapML): %.4f" % r2_score_snapml)

Overwriting /project_data/data_asset/models/snapML-linearRegression-main.py


<a id = "snapml-cpu"></a>
## Step 7 :  Training the SnapML model on CPU with Watson Machine Learning Accelerator
### Re-define the submission parameters

In [21]:
# specify the model file, conda env, device type and device number
args = '--exec-start tensorflow --cs-datastore-meta type=fs \
--workerDeviceNum 1 \
--workerMemory 16G \
--workerDeviceType cpu \
--conda-env-name snapml-177rc \
--model-main ' + model_main
print(args)

--exec-start tensorflow --cs-datastore-meta type=fs --workerDeviceNum 1 --workerMemory 16G --workerDeviceType cpu --conda-env-name snapml-177rc --model-main snapML-linearRegression-main.py


### Submit WMLA Workload

In [22]:
files = {'file': open("{0}/{1}".format(model_dir,model_main),'rb')}
submit_job_to_wmla (args, files)

Refreshing every 5 seconds


,id,args,submissionId,creator,state,appId,schedulerUrl,modelFileOwnerName,workDir,appName,createTime,elastic,nameSpace,numWorker,framework
0,wmla-1274,--exec-start tensorflow --cs-datastore-meta type=fs --workerDeviceNum 1 --workerMemory 16G --workerDeviceType cpu --...,wmla-1274,mluser1,FINISHED,wmla-1274,https://wmla-mss:9080,wmla,/gpfs/myresultfs/mluser1/batchworkdir/wmla-1274/_submitted_code,SingleNodeTensorflowTrain,2021-08-06T07:59:22Z,False,wmla,1,tensorflow


{ 'appId': 'wmla-1274',
  'appName': 'SingleNodeTensorflowTrain',
  'args': '--exec-start tensorflow --cs-datastore-meta type=fs '
          '--workerDeviceNum 1 --workerMemory 16G --workerDeviceType cpu '
          '--conda-env-name snapml-177rc --model-main '
          'snapML-linearRegression-main.py ',
  'createTime': '2021-08-06T07:59:22Z',
  'creator': 'mluser1',
  'elastic': False,
  'framework': 'tensorflow',
  'id': 'wmla-1274',
  'modelFileOwnerName': 'wmla',
  'nameSpace': 'wmla',
  'numWorker': 1,
  'schedulerUrl': 'https://wmla-mss:9080',
  'state': 'FINISHED',
  'submissionId': 'wmla-1274',
  'workDir': '/gpfs/myresultfs/mluser1/batchworkdir/wmla-1274/_submitted_code'}

Totallly training cost:  83  seconds.


<a id = "snapml-model-gpu"></a>
## Step 8 :  Setup Linear Regression model using snapML on GPU
### Create a Linear Regression Model based on snapML 

In [23]:
model_main='snapML-GPU-'+model_base_name

In [24]:
%%writefile {model_dir}/{model_main}

import datetime
#from sklearn.datasets import make_regression
#from sklearn.model_selection import train_test_split
#from sklearn.datasets import make_sparse_uncorrelated
from sklearn.metrics import r2_score
from snapml import LinearRegression as SnapLinearRegression
import numpy as np
from scipy.sparse import load_npz

# load Data
data_dir =  "/gpfs/mydatafs/datasets/dataset-lr"
start = datetime.datetime.now()
X_train = load_npz(data_dir + "/X_train.npz")
X_test  = load_npz(data_dir + "/X_test.npz")
y_train = np.load(data_dir + "/y_train.npy")
y_test = np.load(data_dir + "/y_test.npy")
end = datetime.datetime.now()
print ("load data timecost: %.2gs" % ((end-start).total_seconds()))

print("Number of examples: %d" % (X_train.shape[0]))
print("Number of features: %d" % (X_train.shape[1]))

# snapML model
lr_model = SnapLinearRegression(use_gpu=True,
                                fit_intercept=True,normalize=True)

# Fit
start = datetime.datetime.now()
lr_model.fit(X_train, y_train)
end = datetime.datetime.now()
print ("train timecost: %.2gs" % ((end-start).total_seconds()))

r2_score_snapml = r2_score(y_test, lr_model.predict(X_test))
print("R^2 score (snapML): %.4f" % r2_score_snapml)

Overwriting /project_data/data_asset/models/snapML-GPU-linearRegression-main.py


<a id = "snapml-gpu"></a>
## Step 9 :  Training the SnapML model on GPU with Watson Machine Learning Accelerator
### Re-define the submission parameters

In [25]:
# specify the model file, conda env, device type and device number
args = '--exec-start tensorflow --cs-datastore-meta type=fs \
--workerDeviceNum 1 \
--workerMemory 16G \
--workerDeviceType gpu \
--conda-env-name snapml-177rc \
--model-main ' + model_main
print(args)

--exec-start tensorflow --cs-datastore-meta type=fs --workerDeviceNum 1 --workerMemory 16G --workerDeviceType gpu --conda-env-name snapml-177rc --model-main snapML-GPU-linearRegression-main.py


In [26]:
files = {'file': open("{0}/{1}".format(model_dir,model_main),'rb')}
submit_job_to_wmla (args, files)

Refreshing every 5 seconds


,id,args,submissionId,creator,state,appId,schedulerUrl,modelFileOwnerName,workDir,appName,createTime,elastic,nameSpace,numWorker,framework
0,wmla-1275,--exec-start tensorflow --cs-datastore-meta type=fs --workerDeviceNum 1 --workerMemory 16G --workerDeviceType gpu --...,wmla-1275,mluser1,FINISHED,wmla-1275,https://wmla-mss:9080,wmla,/gpfs/myresultfs/mluser1/batchworkdir/wmla-1275/_submitted_code,SingleNodeTensorflowTrain,2021-08-06T08:00:45Z,False,wmla,1,tensorflow


{ 'appId': 'wmla-1275',
  'appName': 'SingleNodeTensorflowTrain',
  'args': '--exec-start tensorflow --cs-datastore-meta type=fs '
          '--workerDeviceNum 1 --workerMemory 16G --workerDeviceType gpu '
          '--conda-env-name snapml-177rc --model-main '
          'snapML-GPU-linearRegression-main.py ',
  'createTime': '2021-08-06T08:00:45Z',
  'creator': 'mluser1',
  'elastic': False,
  'framework': 'tensorflow',
  'id': 'wmla-1275',
  'modelFileOwnerName': 'wmla',
  'nameSpace': 'wmla',
  'numWorker': 1,
  'schedulerUrl': 'https://wmla-mss:9080',
  'state': 'FINISHED',
  'submissionId': 'wmla-1275',
  'workDir': '/gpfs/myresultfs/mluser1/batchworkdir/wmla-1275/_submitted_code'}

Totallly training cost:  81  seconds.
